# --------------------preprocess--------------------

# Import packages

In [4]:
#python 3.6.13
import pickle
import pandas as pd
import sys
from collections import Counter
from multiprocessing import Pool

import numpy as np
import os
import scipy.signal as sg
import wfdb
from tqdm import tqdm
#install opencv-python==4.5.5.64 first then U can install biosppy
import biosppy

# data

In [16]:
sampling_rate = 250
invalid_labels = ['|', '~', '!', '+', '[', ']', '"', 'x']  # non-beat labels
before = 63
after = 77
tol = 0.05

#write data link
base_dir="C:/Users/ASUS/Desktop/cg working/HRV/羅孟宗老師 已標註波形/羅孟宗老師 已標註波形/"
ECGfile=["完成檔raw data/C783_2016_10_04-15_38_04_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-09_36_26_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-09_59_36_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-10_54_43_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-14_51_20_ECG.csv",\
         "完成檔raw data/C795_2016_11_02-11_43_05_ECG.csv",\
         "完成檔raw data/C795_2016_11_03-09_28_28_ECG.csv",\
         "完成檔raw data/C795_2016_11_03-14_28_24_ECG.csv",\
         "完成檔raw data/C795_2016_11_04-07_48_32_ECG.csv",\
         "完成檔raw data/C807_2016_11_16-11_46_27_ECG.csv",\
         "完成檔raw data/C807_2016_11_18-07_31_57_ECG.csv"]
Markfile=["標註在raw data 上/C783_2016_10_04-15_38_04_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-09_36_26_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-09_59_36_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-10_54_43_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-14_51_20_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_02-11_43_05_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_03-09_28_28_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_03-14_28_24_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_04-07_48_32_True_rr.csv",\
          "標註在raw data 上/C807_2016_11_16-11_46_27_True_rr.csv",\
          "標註在raw data 上/C807_2016_11_18-07_31_57_True_rr.csv"]

#write save link (after preprocess and we will get training data & testing data)
link="C:/Users/ASUS/Desktop/cg working/HRV/Identify correct wave methods/Deep-Neural-Network-For-Heartbeat-Classification-master/"

# preprocess function

In [17]:
def pre_processing(EcgWaveform,Mark):
    filename = EcgWaveform
    signal = pd.read_csv(filename)["EcgWaveform"]
    signal=np.array(list(signal),dtype="f8")

    baseline = sg.medfilt(sg.medfilt(signal, int(0.2 * sampling_rate) - 1), int(0.6 * sampling_rate) - 1)
    signal = signal - baseline

    lab=pd.read_csv(Mark)
    r_peaks=np.array(list(round(lab["position"])),dtype="int")
    
    # align r-peaks
    newR = []
    for r_peak in r_peaks:
        r_left = np.maximum(r_peak - int(tol * sampling_rate), 0)
        r_right = np.minimum(r_peak + int(tol * sampling_rate), len(signal))
        newR.append(r_left + np.argmax(signal[r_left:r_right]))
    r_peaks = np.array(newR, dtype="int")

    # r-peaks intervals
    rris = np.diff(r_peaks)

    avg_rri = np.mean(rris)
    x1, x2 = [], []
    for index in tqdm(range(len(r_peaks)), file=sys.stdout):
        if index == 0 or index == len(r_peaks) - 1:
            continue
        beat = signal[r_peaks[index] - before: r_peaks[index] + after]

        pre_rri = rris[index - 1]
        post_rri = rris[index]
        ratio_rri = pre_rri / post_rri
        local_rri = np.mean(rris[np.maximum(index - 10, 0):index])

        x1.append(beat)
        x2.append([pre_rri - avg_rri, post_rri - avg_rri, ratio_rri, local_rri - avg_rri])

    return x1, x2

# run

In [18]:
x1_train, x2_train = [], []
x1_test, x2_test = [], []
np.random.seed(1225)
for i in range(len(ECGfile)):
    x1,x2=pre_processing(os.path.join(base_dir, ECGfile[i]),os.path.join(base_dir, Markfile[i]))
    a=np.floor(0.8*len(x1[0]))
    if (0.8*len(x1[0])-a)>=0.5:
        a+1
    b=len(x1[0])-a
    a=int(a)
    b=int(b)
    rdgroup=np.random.choice([0]*a+[1]*b,len(x1[0]),replace=False)

    x1tn,x2tn,x1tt,x2tt=[],[],[],[]
    for i in range(len(x1[0])):
        if rdgroup[i]==1:
            x1tt.append(x1[i])
            x2tt.append(x2[i])
        if rdgroup[i]==0:
            x1tn.append(x1[i])
            x2tn.append(x2[i])
    
    x1_train.append(x1tn)
    x2_train.append(x2tn)
    x1_test.append(x1tt)
    x2_test.append(x2tt)

x1_train = np.concatenate(x1_train, axis=0)
x2_train = np.concatenate(x2_train, axis=0)
x1_test = np.concatenate(x1_test, axis=0)
x2_test = np.concatenate(x2_test, axis=0)

with open(os.path.join(link, "CNNMPFL_newwave_preprocess.pkl"), "wb") as f:
        pickle.dump((
            (x1_train, x2_train),
            (x1_test, x2_test)
        ), f, protocol=4)



100%|██████████| 1610/1610 [00:00<00:00, 115747.58it/s]


c:\Users\ASUS\anaconda3\envs\ecg-focalloss\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in long_scalars


100%|██████████| 1921/1921 [00:00<00:00, 442439.07it/s]
